# AIG230 NLP (Week 3 Lab) — Notebook 2: Statistical Language Models (Train, Test, Evaluate)

This notebook focuses on **n-gram Statistical Language Models (SLMs)**:
- Train **unigram**, **bigram**, **trigram** models
- Handle **OOV** with `<UNK>`
- Apply **smoothing** (Add-k)
- Evaluate with **cross-entropy** and **perplexity**
- Do **next-word prediction** and simple **text generation**

> Industry framing: even if modern systems use neural LMs, n-gram LMs are still useful for
baselines, constrained domains, and for understanding evaluation.


### What is smoothing?

Smoothing is a way to stop a language model from saying “this can never happen.”

When we train a language model from data, it only knows what it has seen before.
If it never saw a particular word sequence, the model would normally give it a probability of zero.

Smoothing fixes that.
### Why is this a problem without smoothing?

Imagine the model learned English only by reading a small number of news articles.

If it never saw:

- “oil prices explode”

the model would conclude:

- “That sentence is impossible.”

But as humans, we know it could happen. The model just hasn’t seen it yet.

Without smoothing:

- One unseen word makes the whole sentence probability zero

- Evaluation breaks

- The model is too confident and too brittle

## 0) Setup


In [130]:

import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict


## 1) Data: domain text you might see in real systems


We use short texts that resemble:
- release notes
- incident summaries
- operational runbooks
- customer support messaging

In practice, you would load thousands to millions of lines.


In [131]:

corpus = [
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
    "vpn disconnects frequently after windows update",
    "password reset link expired user cannot login",
    "api requests timeout when latency spikes",
    "portal returns 500 error after deployment",
    "email delivery delayed messages queued",
    "mfa prompt never arrives user stuck at login",
    "wifi drops in meeting rooms access point reboot helps",
    "outlook search not returning results index corrupted",
    "printer driver install fails with error 1603",
    "teams calls choppy audio jitter high",
    "permission denied accessing shared drive though in correct group",
    "battery drains fast after bios update power settings unchanged",
    "push notifications not working on android app",
    "mailbox full cannot receive emails auto archive not running",
]

# Train/test split at sentence level
random.seed(42)
random.shuffle(corpus)
split = int(0.75 * len(corpus))
train_texts = corpus[:split]
test_texts = corpus[split:]

len(train_texts), len(test_texts), train_texts[:2], test_texts[:2]


(21,
 7,
 ['email delivery delayed messages queued',
  'api requests timeout when latency spikes'],
 ['printer driver install fails with error 1603',
  'outlook search not returning results index corrupted'])

## 2) Tokenization + special tokens


We will:
- lowercase
- keep alphanumerics
- split on whitespace
- add sentence boundary tokens: `<s>` and `</s>`

We will also map rare tokens to `<UNK>` based on training frequency.


In [132]:

def tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    #print(text)
    return text.split()

def add_boundaries(tokens: List[str], n: int) -> List[str]:
    # For n-grams, prepend (n-1) start tokens for simpler context handling
    return ["<s>"]*(n-1) + tokens + ["</s>"]

# Example
tokens = tokenize("Printer driver install fails with error 1603")
add_boundaries(tokens, n=3)


['<s>',
 '<s>',
 'printer',
 'driver',
 'install',
 'fails',
 'with',
 'error',
 '1603',
 '</s>']

## 3) Build vocabulary and handle OOV with <UNK>


In [133]:

# Build vocab from training data
train_tokens_flat = []
for t in train_texts:
    train_tokens_flat.extend(tokenize(t))

print(len(train_tokens_flat), train_tokens_flat)

154 ['email', 'delivery', 'delayed', 'messages', 'queued', 'api', 'requests', 'timeout', 'when', 'latency', 'spikes', 'permission', 'denied', 'accessing', 'shared', 'drive', 'though', 'in', 'correct', 'group', 'outlook', 'search', 'not', 'returning', 'results', 'index', 'corrupted', 'mailbox', 'full', 'cannot', 'receive', 'emails', 'auto', 'archive', 'not', 'running', 'wifi', 'drops', 'in', 'meeting', 'rooms', 'access', 'point', 'reboot', 'helps', 'mfa', 'prompt', 'never', 'arrives', 'user', 'stuck', 'at', 'login', 'battery', 'drains', 'fast', 'after', 'bios', 'update', 'power', 'settings', 'unchanged', 'vpn', 'disconnects', 'frequently', 'after', 'windows', 'update', 'teams', 'calls', 'choppy', 'audio', 'jitter', 'high', 'permission', 'denied', 'accessing', 'shared', 'drive', 'though', 'in', 'correct', 'group', 'push', 'notifications', 'not', 'working', 'on', 'android', 'app', 'mfa', 'prompt', 'never', 'arrives', 'user', 'stuck', 'at', 'login', 'password', 'reset', 'link', 'expired', 

In [134]:
freq = Counter(train_tokens_flat)

In [135]:
len(freq.items())

85

In [150]:
count = 0
word_count = 2
for w,c in freq.items():
    if c>= word_count:
        count += 1
        #print (w,c)

print (f'Tot words: {count} that frequency >= {word_count} out of {len(freq)} words')

Tot words: 56 that frequency >= 2 out of 85 words


In [137]:

# Typical practical rule: map tokens with frequency <= 1 to <UNK> in small corpora
min_count = word_count
vocab = {w for w, c in freq.items() if c >= min_count}
vocab |= {"<UNK>", "<s>", "</s>"}

In [138]:
print(len(vocab))
vocab

88


{'500',
 '</s>',
 '<UNK>',
 '<s>',
 'access',
 'accessing',
 'after',
 'android',
 'api',
 'app',
 'archive',
 'arrives',
 'at',
 'audio',
 'auto',
 'battery',
 'bios',
 'calls',
 'cannot',
 'choppy',
 'correct',
 'corrupted',
 'delayed',
 'delivery',
 'denied',
 'deployment',
 'disconnects',
 'drains',
 'drive',
 'drops',
 'email',
 'emails',
 'error',
 'expired',
 'fast',
 'frequently',
 'full',
 'group',
 'helps',
 'high',
 'in',
 'index',
 'jitter',
 'latency',
 'link',
 'login',
 'mailbox',
 'meeting',
 'messages',
 'mfa',
 'never',
 'not',
 'notifications',
 'on',
 'outlook',
 'password',
 'permission',
 'point',
 'portal',
 'power',
 'prompt',
 'push',
 'queued',
 'reboot',
 'receive',
 'requests',
 'reset',
 'results',
 'returning',
 'returns',
 'rooms',
 'running',
 'search',
 'settings',
 'shared',
 'spikes',
 'stuck',
 'teams',
 'though',
 'timeout',
 'unchanged',
 'update',
 'user',
 'vpn',
 'when',
 'wifi',
 'windows',
 'working'}

In [139]:

def replace_oov(tokens: List[str], vocab: set) -> List[str]:
    return [tok if tok in vocab else "<UNK>" for tok in tokens]

# Show OOV effect
sample = tokenize(test_texts[0])
sample, replace_oov(sample, vocab)


(['printer', 'driver', 'install', 'fails', 'with', 'error', '1603'],
 ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'error', '<UNK>'])

## 4) Train n-gram counts (unigram, bigram, trigram)


We will compute:
- `ngram_counts[(w1,...,wn)]`
- `context_counts[(w1,...,w_{n-1})]`

Then probability:
\ndefault:  P(w_n | context) = count(context + w_n) / count(context)

This fails when an n-gram is unseen, so we add smoothing.


In [140]:
def get_ngrams(tokens: List[str], n: int):
    return[tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def train_ngram_counts(texts: List[str], n: int, vocab: set):
    ngrams_counts = Counter()
    context_counts = Counter()

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n=n)

        #print(toks)

        for ng in get_ngrams(toks, n):
            ngrams_counts[ng] += 1
            context = ng[:-1]
            context_counts[context] += 1
    return ngrams_counts, context_counts

In [141]:
uni_counts, uni_ctx = train_ngram_counts(train_texts, n=1, vocab=vocab)
bi_counts, bi_ctx   = train_ngram_counts(train_texts, n=2, vocab=vocab)
tri_counts, tri_ctx = train_ngram_counts(train_texts, n=3, vocab=vocab)

print('uni ',len(uni_counts), len(uni_ctx))
print('bi ',len(bi_counts), len(bi_ctx))
print('tri',len(tri_counts), len(tri_ctx))

uni  86 1
bi  106 86
tri 107 95


In [142]:
bi_ctx

Counter({('<s>',): 21,
         ('not',): 5,
         ('cannot',): 4,
         ('user',): 4,
         ('login',): 4,
         ('after',): 4,
         ('in',): 3,
         ('update',): 3,
         ('email',): 2,
         ('delivery',): 2,
         ('delayed',): 2,
         ('messages',): 2,
         ('queued',): 2,
         ('api',): 2,
         ('requests',): 2,
         ('timeout',): 2,
         ('when',): 2,
         ('latency',): 2,
         ('spikes',): 2,
         ('permission',): 2,
         ('denied',): 2,
         ('accessing',): 2,
         ('shared',): 2,
         ('drive',): 2,
         ('though',): 2,
         ('correct',): 2,
         ('group',): 2,
         ('mailbox',): 2,
         ('full',): 2,
         ('receive',): 2,
         ('emails',): 2,
         ('auto',): 2,
         ('archive',): 2,
         ('running',): 2,
         ('mfa',): 2,
         ('prompt',): 2,
         ('never',): 2,
         ('arrives',): 2,
         ('stuck',): 2,
         ('at',): 2,
         ('b

## 5) Add-k smoothing and probability function


### What does Add-k smoothing do?
Add-k smoothing tells the model:

- “Even if you didn’t see something, assume it could still happen a little bit.”

It does this by:

- Giving every possible next word a tiny amount of probability

- Not just the ones seen in training

So instead of:

- seen → possible

- unseen → impossible

We get:

- seen → more likely

- unseen → less likely, but still possible


### Why is it called Add-k?

Because we add a small number k to every word count.

Think of it as:

- adding a tiny “imaginary observation” for every word

- so no word ever has zero probability

When k is small (like 0.1 or 0.5), it gently smooths the probabilities instead of overpowering real data.

In [143]:
# This function calculates the probability of a word appearing next, given the previous words, while making sure the probability is never zero.
def prob_addk(ngram: Tuple[str, ...], ngram_counts: Counter, context_counts: Counter, V: int, k: float = 0.5) -> float:
    context = ngram[:-1]  #“Give me everything except the last element”
    return (ngram_counts[ngram] + k) / (context_counts[context] + k * V)

V = len(vocab)
# Example: P("login" | "<s>") in bigram model
# How likely is the word "login" to come after "<s>"?
example = ("<s>", "gold")
prob_addk(example, bi_counts, bi_ctx, V, k=0.5)

0.007692307692307693

In [144]:
def prob_backoff(trigram,
            tri_counts, tri_ctx,
            bi_counts, bi_ctx,
            uni_counts, uni_ctx,
            vocab,
            k=0.5
            ):
    w1, w2, w3 = trigram
    V = len(vocab)

    if tri_counts[trigram] > 0 and tri_ctx[(w1,w2)] > 0:
        return prob_addk(trigram, tri_counts, tri_ctx, V, k), 'tri'

    bigram = (w2,w3)

    if bi_counts[bigram] > 0 and bi_ctx[(w2,)] > 0:
        return prob_addk(bigram, bi_counts, bi_ctx, V, k), 'bi'

    uni = (w1,)
    return prob_addk(uni, uni_counts, uni_ctx, V, k), 'uni'


In [145]:
example = ("vpn" , "disconnects",  "frequently")
prob_backoff(example,tri_counts, tri_ctx, bi_counts, bi_ctx, uni_counts, uni_ctx, vocab, k=0.5)

(0.03333333333333333, 'tri')

## 6) Evaluate: cross-entropy and perplexity on test set


We evaluate an LM by how well it predicts held-out text.

Cross-entropy (average negative log probability):
H = - (1/N) * sum log2 P(w_i | context)

Perplexity:
PP = 2^H

Lower perplexity is better.


In [146]:
def evaluate_perplexity(texts: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k: float = 0.5) -> float:
    V = len(vocab)
    log2_probs = []
    token_count = 0

    for text in texts:
        toks = replace_oov(tokenize(text), vocab)
        toks = add_boundaries(toks, n)
        ngrams = get_ngrams(toks, n)
        for ng in ngrams:
            p = prob_addk(ng, ngram_counts, context_counts, V, k=k)
            log2_probs.append(math.log(p, 2))
            token_count += 1

    H = -sum(log2_probs) / token_count
    PP = 2 ** H
    return PP

pp_uni = evaluate_perplexity(test_texts, n=1, ngram_counts=uni_counts, context_counts=uni_ctx, vocab=vocab, k=0.5)
pp_bi  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=0.5)
pp_tri = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=0.5)

pp_uni, pp_bi, pp_tri

(120.30682300277988, 43.30067859488039, 43.029015025061675)

## 7) Next-word prediction (top-k)


Given a context, compute the probability of each candidate next token and return the top-k.

This mirrors:
- autocomplete in constrained domains
- template suggestion systems
- command prediction in runbooks


In [147]:
def next_word_topk(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5, top_k: int = 5):
    # Context length should be n-1
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    candidates = []
    for w in vocab:
        if w in {"<s>"}:
            continue
        ng = context + (w,)
        p = prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth)
        candidates.append((w, p))
    candidates.sort(key=lambda x: -x[1])
    return candidates[:top_k]

# Bigram: context is 1 token
next_word_topk(["<s>"], n=2, ngram_counts=bi_counts, context_counts=bi_ctx, vocab=vocab, top_k=8)




[('battery', 0.038461538461538464),
 ('email', 0.038461538461538464),
 ('push', 0.038461538461538464),
 ('password', 0.038461538461538464),
 ('mfa', 0.038461538461538464),
 ('permission', 0.038461538461538464),
 ('api', 0.038461538461538464),
 ('mailbox', 0.038461538461538464)]

In [151]:
#Create a function that returns top-5 next words given a phrase like: "user cannot".

phrase= ['user', 'cannot']
next_word_topk(phrase, n=2, ngram_counts=bi_counts, context_counts=bi_ctx, vocab=vocab)

[('receive', 0.052083333333333336),
 ('login', 0.052083333333333336),
 ('battery', 0.010416666666666666),
 ('update', 0.010416666666666666),
 ('500', 0.010416666666666666)]

## 8) Simple generation (bigram or trigram)


Text generation is not the main goal in SLMs, but it helps you verify:
- boundary handling
- smoothing
- OOV decisions

We will sample tokens until we hit `</s>`.


In [148]:

def sample_next(context_tokens: List[str], n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, k_smooth: float = 0.5):
    V = len(vocab)
    context = tuple(context_tokens[-(n-1):]) if n > 1 else tuple()
    words = [w for w in vocab if w != "<s>"]
    probs = []
    for w in words:
        ng = context + (w,)
        probs.append(prob_addk(ng, ngram_counts, context_counts, V, k=k_smooth))
    # Normalize
    s = sum(probs)
    probs = [p/s for p in probs]
    return random.choices(words, weights=probs, k=1)[0]

def generate(n: int, ngram_counts: Counter, context_counts: Counter, vocab: set, max_len: int = 20, k_smooth: float = 0.5):
    tokens = ["<s>"]*(n-1) if n > 1 else []
    out = []
    for _ in range(max_len):
        w = sample_next(tokens, n, ngram_counts, context_counts, vocab, k_smooth=k_smooth)
        if w == "</s>":
            break
        out.append(w)
        tokens.append(w)
    return " ".join(out)

for _ in range(5):
    print("BIGRAM:", generate(2, bi_counts, bi_ctx, vocab, max_len=18))


BIGRAM: high permission battery on android delayed rooms reboot unchanged point point fast requests on android notifications search after
BIGRAM: messages user requests error <UNK> settings spikes
BIGRAM: deployment correct power email power choppy delayed after
BIGRAM: though mailbox when full accessing teams rooms returns corrupted when queued drops group running
BIGRAM: battery meeting power


## 9) Model comparison: effect of n and smoothing


Try different `k` values. Notes:
- `k=1.0` is Laplace smoothing (often too strong)
- smaller `k` (like 0.1 to 0.5) is often better

In real corpora, trigrams often beat bigrams, but require more data.


In [149]:


for k in [1.0, 0.5, 0.1, 0.01]:
    pp_bi_k  = evaluate_perplexity(test_texts, n=2, ngram_counts=bi_counts,  context_counts=bi_ctx,  vocab=vocab, k=k)
    pp_tri_k = evaluate_perplexity(test_texts, n=3, ngram_counts=tri_counts, context_counts=tri_ctx, vocab=vocab, k=k)
    print(f"k={k:>4}:  bigram PP={pp_bi_k:,.2f}   trigram PP={pp_tri_k:,.2f}")




k= 1.0:  bigram PP=55.88   trigram PP=55.70
k= 0.5:  bigram PP=43.30   trigram PP=43.03
k= 0.1:  bigram PP=20.60   trigram PP=20.06
k=0.01:  bigram PP=8.96   trigram PP=8.03


## Exercises (do these during lab)
1) Add 20 more realistic domain sentences to the corpus and re-run training/evaluation.  
2) Change `min_count` (OOV threshold) and explain how perplexity changes.  
3) Implement **backoff**: if a trigram is unseen, fall back to bigram; if unseen,
4) Create a function that returns top-5 next words given a phrase like: "user cannot".
